앞으로 실행할 명령들은 jupyter에서 async error가 발생한다.
https://stackoverflow.com/questions/61926359/django-synchronousonlyoperation-you-cannot-call-this-from-an-async-context-u 을 참고해서 아래와 같이 설정하자.

In [1]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
from django.contrib.auth import get_user_model
from django.db.models import Q
from events.models import *
from entities.models import *
User = get_user_model()

## 5. Union 연산
- Union 연산을 이용해 쿼리셋을 합할 수 있다.
- Union all은 중복을 고려하지 않고 쿼리셋을 합한다.

In [2]:
q1 = User.objects.filter(id__gte=5)
print(q1)

<QuerySet [<User: sohan>, <User: Raghu>, <User: rishab>, <User: yash>, <User: John>, <User: Ricky>, <User: sharukh>]>


In [3]:
q2 = User.objects.filter(id__lte=9)
print(q2)

<QuerySet [<User: admin>, <User: Ritesh>, <User: Billy>, <User: Radha>, <User: sohan>, <User: Raghu>, <User: rishab>, <User: yash>, <User: John>]>


In [4]:
print(q1.union(q2))

<QuerySet [<User: admin>, <User: Ritesh>, <User: Billy>, <User: Radha>, <User: sohan>, <User: Raghu>, <User: rishab>, <User: yash>, <User: John>, <User: Ricky>, <User: sharukh>]>


In [5]:
print(q2.union(q1))

<QuerySet [<User: admin>, <User: Ritesh>, <User: Billy>, <User: Radha>, <User: sohan>, <User: Raghu>, <User: rishab>, <User: yash>, <User: John>, <User: Ricky>, <User: sharukh>]>


In [6]:
q3 = EventVillain.objects.all()
print(q3)

<QuerySet []>


- 필드와 데이터 유형이 다른 User와 EventVillain에 Union을 수행하면 에러가 발생한다

In [7]:
# 에러 발생
try:
    q1.union(q3)
except:
    print('에러 발생')

- 다른 모델이라도 공통적으로 가지고 있는 필드를 쿼리해온다면 Union 연산을 수행할 수 있다.
- Hero모델과 Villain모델은 공통적으로 name과 gender를 가지고 있다.

In [8]:
Hero.objects.all().values_list(
    "name", "gender"
).union(
Villain.objects.all().values_list(
    "name", "gender"
))

<QuerySet []>

## 6. 필요한 필드만 조회하는 방법
- 쿼리셋의 `values`메서드와 `values_list` 메서드를 이용한다.
- `only` 메서드를 이용한다.

In [9]:
queryset = User.objects.filter(
    first_name__startswith='R'
).values(
    'first_name', 
    'last_name'
)
queryset

<QuerySet [{'first_name': 'Ritesh', 'last_name': 'Kim'}, {'first_name': 'Radha', 'last_name': 'Do'}, {'first_name': 'Raghu', 'last_name': 'Ga'}, {'first_name': 'rishab', 'last_name': 'bill'}, {'first_name': 'Ricky', 'last_name': 'Ari'}]>

In [10]:
str(queryset.query)

'SELECT "auth_user"."first_name", "auth_user"."last_name" FROM "auth_user" WHERE "auth_user"."first_name" LIKE R% ESCAPE \'\\\''

In [11]:
queryset = User.objects.filter(
    first_name__startswith='R'
).only('first_name', 'last_name')
queryset

<QuerySet [<User: Ritesh>, <User: Radha>, <User: Raghu>, <User: rishab>, <User: Ricky>]>

In [12]:
print(str(queryset.query))

SELECT "auth_user"."id", "auth_user"."first_name", "auth_user"."last_name" FROM "auth_user" WHERE "auth_user"."first_name" LIKE R% ESCAPE '\'


:bangbang: `only`메서드가 `values`메서드는 `id`필드를 가져오냐 안 가져오냐 차이이다.